In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

pd.set_option('display.max_columns', None)
DATA_PATH = './data/'
pd.set_option('display.max_rows', 500)

# load model
clf = joblib.load('model')

In [17]:
league_data = pd.read_csv('data/test_data_part1.csv')

# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
league_data.MW = league_data.MW.astype(float)

for col in cols:
    league_data[col] = league_data[col] / league_data.MW
    
league_data.tail()

,Unnamed: 0,Date,season,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP,B365H,B365D,B365A,HM1,HM2,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5,MW,HomeTeamLP,AwayTeamLP,gameId,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
159,159,2021-01-02,2021,Brighton,Wolves,3,3,D,17,13,22,21,0.812500,1.312500,2.62,3.25,2.75,D,D,D,L,L,D,L,W,L,L,16.0,15,7,160,DDDLL,DLWLL,3,4,-0.312500,-0.500000,-0.500000,-0.062500,8
160,160,2021-01-02,2021,West Brom,Arsenal,0,4,A,11,17,36,19,0.470588,1.176471,5.75,4.00,1.57,L,D,L,D,L,W,W,L,D,L,17.0,18,8,161,LDLDL,WWLDL,2,7,-1.470588,-0.117647,-0.705882,-0.294118,10
161,161,2021-01-03,2021,Newcastle,Leicester,1,2,A,16,28,26,21,1.117647,1.705882,5.25,4.20,1.61,L,D,L,D,L,D,D,W,L,W,17.0,13,5,162,LDLDL,DDWLW,2,8,-0.588235,0.411765,-0.588235,-0.352941,8
162,162,2021-01-03,2021,Chelsea,Man City,1,3,A,29,21,20,12,1.529412,1.705882,3.00,3.70,2.20,D,L,W,L,L,M,W,W,W,D,17.0,4,2,163,DLWLL,MWWWD,4,10,0.529412,0.529412,-0.176471,-0.352941,2
163,163,2021-01-04,2021,Southampton,Liverpool,1,0,H,26,33,18,18,1.529412,1.941176,5.00,4.33,1.61,D,D,L,D,W,D,D,W,W,D,17.0,11,1,164,DDLDW,DDWWD,6,9,0.470588,0.882353,-0.411765,-0.176471,10


In [18]:
league_data.drop(['Unnamed: 0','FTHG', 'FTAG', 'Date', 'HomeTeam', 'AwayTeam', 'HomeTeamLP','AwayTeamLP',
                 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HM1', 'HM2', 'HM3', 'MW','HTGS','ATGS','HTGC','ATGC',
                 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5', 'HTFormPtsStr','HTFormPts','ATFormPts',
                 'ATFormPtsStr'], 1, inplace=True)
league_data.columns

Index(['season', 'FTR', 'HTP', 'ATP', 'B365H', 'B365D', 'B365A', 'gameId',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'],
      dtype='object')

In [19]:
EMA_data = pd.read_csv('data/test_data_part2.csv')
EMA_data.tail()

,Unnamed: 0,f_DateHome,f_seasonHome,gameId,HomeTeam,homeGame_x,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_halfTimeGoalsAgainstHome,f_halfTimeGoalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,f_shotsOnTargetAgainstHome,f_shotsOnTargetForHome,f_yellowsAgainstHome,f_yellowsForHome,f_DateAway,f_seasonAway,AwayTeam,homeGame_y,f_cornersAgainstAway,f_cornersForAway,f_freesAgainstAway,f_freesForAway,f_goalsAgainstAway,f_goalsForAway,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,f_yellowsAgainstAway,f_yellowsForAway
137,159,2021-02-01,2021,159,Brighton,1,4.046708,5.747446,10.369829,11.872098,1.509318,1.050347,0.588747,0.461302,0.076573,0.163491,8.870309,12.702409,3.238342,3.544956,1.508949,1.340342,2021-02-01,2021,Wolves,0,4.738488,4.978083,10.871905,12.855069,1.314873,0.911116,0.493740,0.337284,0.057871,0.000000,10.603202,12.157255,3.299475,4.327570,1.763185,1.601408
138,160,2021-02-01,2021,160,West Brom,1,7.278968,3.768354,11.271871,10.711609,2.197005,0.655417,1.017676,0.402053,0.000000,0.191866,16.195490,8.052705,6.136796,2.624080,1.654027,1.347034,2021-02-01,2021,Arsenal,0,5.209125,5.400823,11.577425,9.712271,1.179794,0.978600,0.752703,0.443435,0.000000,0.206947,12.266580,11.069004,3.286768,3.508707,2.084237,1.519142
139,161,2021-03-01,2021,161,Newcastle,1,6.399954,3.928616,11.934246,10.284487,1.596250,1.090854,0.590204,0.265357,0.131890,0.000000,15.035359,9.049679,5.628441,3.306036,1.881200,1.883024,2021-03-01,2021,Leicester,0,4.971831,5.144724,12.867124,9.703660,1.234632,1.728593,0.676410,0.708908,0.000000,0.000000,10.591852,12.123519,4.189759,3.974057,1.580536,2.026649
140,162,2021-03-01,2021,162,Chelsea,1,5.204522,6.153365,9.279909,12.316751,1.116503,1.859677,0.559589,0.620829,0.000000,0.047379,9.591909,14.220644,2.924908,5.309977,1.415966,1.308365,2021-03-01,2021,Man City,0,3.450434,6.869596,9.942586,10.033976,0.757358,1.481119,0.345840,1.065190,0.000000,0.000000,7.900795,15.406108,2.437667,5.329999,1.382368,1.483502
141,163,2021-04-01,2021,163,Southampton,1,5.189161,4.394514,12.657861,12.433748,1.092791,1.484782,0.344229,0.889156,0.129172,0.000000,9.890132,10.222548,3.932677,4.273505,1.545654,1.514391,2021-04-01,2021,Liverpool,0,3.359262,6.584086,9.781757,9.866738,1.114910,2.279970,0.717041,1.089927,0.100800,0.000000,8.353671,14.933519,3.449155,6.228088,1.772083,0.815252


In [20]:
EMA_data.drop(['Unnamed: 0', 'f_DateHome', 'f_seasonHome', 'HomeTeam',
               'homeGame_x', 'f_yellowsAgainstAway', 'f_yellowsForAway',
               'f_yellowsAgainstHome', 'f_yellowsForHome', 'f_DateAway', 'f_seasonAway', 
               'AwayTeam', 'homeGame_y'], 1, inplace=True)
EMA_data.columns

Index(['gameId', 'f_cornersAgainstHome', 'f_cornersForHome',
       'f_freesAgainstHome', 'f_freesForHome', 'f_goalsAgainstHome',
       'f_goalsForHome', 'f_halfTimeGoalsAgainstHome',
       'f_halfTimeGoalsForHome', 'f_redsAgainstHome', 'f_redsForHome',
       'f_shotsAgainstHome', 'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_halfTimeGoalsAgainstAway',
       'f_halfTimeGoalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway'],
      dtype='object')

In [21]:
df = pd.merge(EMA_data, league_data, left_on='gameId', right_index=True)
df.head()
df.drop(['season','gameId','gameId_x','gameId_y'], 1, inplace=True)

df.columns

,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_halfTimeGoalsAgainstHome,f_halfTimeGoalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,f_shotsOnTargetAgainstHome,f_shotsOnTargetForHome,f_cornersAgainstAway,f_cornersForAway,f_freesAgainstAway,f_freesForAway,f_goalsAgainstAway,f_goalsForAway,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,FTR,HTP,ATP,B365H,B365D,B365A,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
137,4.046708,5.747446,10.369829,11.872098,1.509318,1.050347,0.588747,0.461302,0.076573,0.163491,8.870309,12.702409,3.238342,3.544956,4.738488,4.978083,10.871905,12.855069,1.314873,0.911116,0.493740,0.337284,0.057871,0.000000,10.603202,12.157255,3.299475,4.327570,D,0.812500,1.312500,2.62,3.25,2.75,-0.312500,-0.500000,-0.500000,-0.062500,8
138,7.278968,3.768354,11.271871,10.711609,2.197005,0.655417,1.017676,0.402053,0.000000,0.191866,16.195490,8.052705,6.136796,2.624080,5.209125,5.400823,11.577425,9.712271,1.179794,0.978600,0.752703,0.443435,0.000000,0.206947,12.266580,11.069004,3.286768,3.508707,A,0.470588,1.176471,5.75,4.00,1.57,-1.470588,-0.117647,-0.705882,-0.294118,10
139,6.399954,3.928616,11.934246,10.284487,1.596250,1.090854,0.590204,0.265357,0.131890,0.000000,15.035359,9.049679,5.628441,3.306036,4.971831,5.144724,12.867124,9.703660,1.234632,1.728593,0.676410,0.708908,0.000000,0.000000,10.591852,12.123519,4.189759,3.974057,A,1.117647,1.705882,5.25,4.20,1.61,-0.588235,0.411765,-0.588235,-0.352941,8
140,5.204522,6.153365,9.279909,12.316751,1.116503,1.859677,0.559589,0.620829,0.000000,0.047379,9.591909,14.220644,2.924908,5.309977,3.450434,6.869596,9.942586,10.033976,0.757358,1.481119,0.345840,1.065190,0.000000,0.000000,7.900795,15.406108,2.437667,5.329999,A,1.529412,1.705882,3.00,3.70,2.20,0.529412,0.529412,-0.176471,-0.352941,2
141,5.189161,4.394514,12.657861,12.433748,1.092791,1.484782,0.344229,0.889156,0.129172,0.000000,9.890132,10.222548,3.932677,4.273505,3.359262,6.584086,9.781757,9.866738,1.114910,2.279970,0.717041,1.089927,0.100800,0.000000,8.353671,14.933519,3.449155,6.228088,H,1.529412,1.941176,5.00,4.33,1.61,0.470588,0.882353,-0.411765,-0.176471,10


In [32]:
len(['f_cornersAgainstHome', 'f_cornersForHome', 'f_freesAgainstHome',
       'f_freesForHome', 'f_goalsAgainstHome', 'f_goalsForHome',
       'f_halfTimeGoalsAgainstHome', 'f_halfTimeGoalsForHome',
       'f_redsAgainstHome', 'f_redsForHome', 'f_shotsAgainstHome',
       'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_halfTimeGoalsAgainstAway',
       'f_halfTimeGoalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'HTP', 'ATP', 'B365H', 'B365D', 'B365A',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'])

38

In [31]:
len(['f_cornersAgainstHome', 'f_cornersForHome', 'f_freesAgainstHome',
       'f_freesForHome', 'f_goalsAgainstHome', 'f_goalsForHome',
       'f_halfTimeGoalsAgainstHome', 'f_halfTimeGoalsForHome',
       'f_redsAgainstHome', 'f_redsForHome', 'f_shotsAgainstHome',
       'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_halfTimeGoalsAgainstAway',
       'f_halfTimeGoalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'FTR', 'HTP', 'ATP', 'B365H', 'B365D',
       'B365A', 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'])

39

In [ ]:
['f_cornersAgainstHome', 'f_cornersForHome', 'f_freesAgainstHome',
       'f_freesForHome', 'f_goalsAgainstHome', 'f_goalsForHome',
       'f_halfTimeGoalsAgainstHome', 'f_halfTimeGoalsForHome',
       'f_redsAgainstHome', 'f_redsForHome', 'f_shotsAgainstHome',
       'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_halfTimeGoalsAgainstAway',
       'f_halfTimeGoalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'HTP', 'ATP', 'B365H', 'B365D', 'B365A',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP']

In [24]:
X = df.drop(['FTR'], 1)
Y = df['FTR']
def transform_results(results):
    transformed = []
    for i in range(len(results)):
        if results[i] == 'H':
            transformed.append(0)
        elif results[i] == 'A':
            transformed.append(2)
        else:
            transformed.append(1)
    return np.array(transformed)

Y = transform_results(Y)


In [25]:
y_preds = clf.predict_proba(X)
y_pred_argmax = [np.argmax(i) for i in y_preds]
labels = ['Home', 'Draw', 'Away']
print(classification_report(Y, y_pred_argmax, target_names=labels))
print(confusion_matrix(Y, y_pred_argmax))

              precision    recall  f1-score   support

        Home       0.54      0.70      0.61        54
        Draw       0.33      0.28      0.30        40
        Away       0.59      0.48      0.53        48

    accuracy                           0.51       142
   macro avg       0.49      0.49      0.48       142
weighted avg       0.50      0.51      0.50       142

[[38 10  6]
 [19 11 10]
 [13 12 23]]


In [26]:
y_pred_argmax

[2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 2,
 0,
 1]

In [90]:
Y

array([2, 1, 2, 1, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 1, 1, 0, 2, 1, 1,
       1, 2, 1, 2, 2, 1, 2, 1, 0, 0, 1, 2, 1, 2, 0, 2, 2, 0, 2, 0, 2, 0,
       0, 2, 1, 0, 2, 0, 0, 0, 2, 0, 1, 2, 2, 2, 0, 0, 2, 2, 1, 0, 0, 1,
       2, 1, 0, 2, 0, 2, 1, 2, 2, 0, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0,
       1, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 1, 0, 1, 1, 2, 2, 2, 0, 1, 1,
       2, 0, 2, 0, 0, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1, 2, 0, 1, 2, 0,
       1, 2, 0, 0, 0, 1, 2, 2, 2, 0])

In [92]:
df[0:200]

,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_halfTimeGoalsAgainstHome,f_halfTimeGoalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,f_shotsOnTargetAgainstHome,f_shotsOnTargetForHome,f_cornersAgainstAway,f_cornersForAway,f_freesAgainstAway,f_freesForAway,f_goalsAgainstAway,f_goalsForAway,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,FTR,HTP,ATP,B365H,B365D,B365A,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,6.060000,4.960000,12.020000,11.960000,0.510000,2.020000,0.000000,1.000000,0.000000,0.000000,11.040000,9.590000,4.490000,4.020000,2.960000,7.080000,12.960000,8.020000,1.020000,3.040000,0.510000,1.020000,0.510000,0.000000,7.470000,16.020000,4.490000,5.530000,A,1.500000,1.500000,3.75,3.25,2.10,1.000000,1.500000,0.000000,0.000000,6
1,8.060000,1.490000,9.000000,11.490000,4.020000,1.020000,1.020000,0.510000,0.000000,0.510000,15.040000,6.490000,7.000000,2.530000,7.570000,1.980000,6.980000,11.470000,1.510000,1.470000,0.000000,0.490000,0.000000,0.510000,15.550000,7.450000,4.530000,3.980000,D,0.000000,1.000000,9.00,4.75,1.36,-1.000000,-0.666667,-1.000000,-1.000000,14
2,7.550000,7.920000,9.550000,13.000000,1.490000,0.000000,0.980000,0.000000,0.000000,0.510000,14.570000,6.450000,2.980000,1.490000,5.940000,2.550000,13.590000,9.570000,3.490000,3.510000,1.980000,2.000000,0.000000,0.000000,17.920000,8.040000,6.000000,5.040000,A,0.000000,1.000000,2.70,3.30,2.62,-0.666667,0.666667,-1.000000,-1.000000,-9
3,3.000000,3.470000,11.590000,16.530000,1.510000,2.550000,0.510000,0.510000,0.000000,0.000000,14.490000,9.000000,5.530000,5.510000,4.430000,7.000000,14.020000,11.590000,1.530000,0.980000,1.020000,0.000000,0.510000,0.000000,13.980000,10.410000,4.530000,0.980000,D,1.000000,1.000000,1.45,4.50,7.00,1.000000,-1.000000,0.000000,0.000000,-7
4,7.000000,6.470000,9.040000,13.000000,2.000000,0.510000,0.510000,0.510000,0.000000,0.000000,10.920000,14.490000,2.510000,3.000000,8.940000,4.490000,10.450000,6.490000,1.530000,1.490000,1.020000,0.980000,0.000000,0.000000,11.550000,10.490000,5.570000,2.470000,H,0.000000,1.000000,3.60,3.50,2.05,1.000000,-2.000000,-1.000000,-1.000000,9
5,0.510000,10.020000,8.040000,7.470000,1.470000,2.980000,0.980000,1.470000,0.510000,0.000000,5.490000,19.960000,3.000000,6.000000,3.530000,5.040000,12.510000,11.490000,0.510000,2.490000,0.510000,1.000000,0.000000,0.000000,9.590000,9.940000,2.510000,4.470000,H,2.000000,2.000000,1.45,4.75,6.50,1.333333,-0.333333,0.000000,0.000000,-7
6,5.985335,4.414211,7.333689,13.040795,2.026663,2.000533,1.040261,0.320091,0.000000,0.333156,13.278763,12.495267,3.999467,6.067458,6.039195,5.667378,13.053326,13.360885,1.026663,1.666311,0.693508,1.000000,0.000000,0.000000,10.679376,9.038662,4.666844,2.972804,H,1.000000,2.000000,1.40,4.75,8.00,-0.666667,0.333333,-1.000000,-1.000000,-14
7,4.360885,6.705506,14.014131,10.440341,1.013065,2.679376,0.679909,1.359819,0.333156,0.000000,7.653779,13.932542,3.279829,5.346221,3.639115,4.040795,15.039728,13.799627,2.000533,2.013065,0.666844,1.013065,0.000000,0.333156,7.627116,14.733769,2.640715,4.692974,H,2.000000,1.000000,1.72,3.75,4.80,1.333333,0.666667,1.000000,1.000000,-3
8,5.614051,4.093054,15.119184,7.638582,2.279829,2.639648,1.293428,1.306492,0.000000,0.000000,16.560725,11.146914,5.306492,6.413145,4.000000,4.470000,7.020000,10.550000,3.040000,2.490000,0.510000,1.490000,0.000000,0.000000,8.470000,15.020000,4.060000,6.960000,D,1.500000,1.000000,7.50,5.25,1.36,0.500000,-0.500000,0.500000,0.500000,16
9,6.361418,5.959739,14.359819,10.691908,1.346221,0.986935,1.013065,0.000000,0.333156,0.000000,17.107719,8.880816,7.120251,0.986935,8.370000,6.040000,10.490000,10.450000,2.470000,0.980000,1.000000,0.490000,0.000000,0.000000,9.410000,12.940000,3.450000,3.470000,H,1.000000,0.000000,2.40,3.20,3.10,-0.250000,-0.750000,1.000000,1.000000,3


In [99]:
testdf = df.drop(['f_cornersAgainstHome', 'f_cornersForHome', 'f_freesAgainstHome',
       'f_freesForHome', 'f_goalsAgainstHome', 'f_goalsForHome',
       'f_halfTimeGoalsAgainstHome', 'f_halfTimeGoalsForHome',
       'f_redsAgainstHome', 'f_redsForHome', 'f_shotsAgainstHome',
       'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_cornersAgainstAway', 'f_cornersForAway',
       'f_freesAgainstAway', 'f_freesForAway', 'f_goalsAgainstAway',
       'f_goalsForAway', 'f_halfTimeGoalsAgainstAway',
       'f_halfTimeGoalsForAway', 'f_redsAgainstAway', 'f_redsForAway',
       'f_shotsAgainstAway', 'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'FTR', 'HTP', 'ATP',  'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'], 1)

In [101]:
testdf['wynik'] = Y

In [104]:
testdf['predict'] = y_pred_argmax

In [105]:
testdf

,B365H,B365D,B365A,wynik,predict
0,3.75,3.25,2.10,2,2
1,9.00,4.75,1.36,1,2
2,2.70,3.30,2.62,2,2
3,1.45,4.50,7.00,1,0
4,3.60,3.50,2.05,0,0
5,1.45,4.75,6.50,0,0
6,1.40,4.75,8.00,0,2
7,1.72,3.75,4.80,0,0
8,7.50,5.25,1.36,1,2
9,2.40,3.20,3.10,0,1


In [106]:
1.2*1.2*1.2*1.3*1.3*1.4*1.2*1.2*1.2*1.2*1.2*1.5*1.5*1.2

27.468090703872